In [28]:
from gensim.models.keyedvectors import KeyedVectors
from scipy.spatial import distance
import numpy as np
import spacy
from collections import Counter
!spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [16]:
def cosine_similarity(first_word, second_word):
    return 1 - distance.cosine(first_word, second_word)

In [17]:
glove_vectors = KeyedVectors.load_word2vec_format("glove-wiki-gigaword-300.gz", binary=False)  # Make sure to download vectors first

In [33]:
import gensim.downloader as api
glove_vectors = api.load("glove-wiki-gigaword-50")

In [34]:
dpt_store_words = ["clothing", "technology", "appliances", "homeware", "shoes", "accessories", "gifts"]

In [19]:
dpt_store_centroid = np.zeros(300)

In [20]:
for word in dpt_store_words:
    dpt_store_centroid += glove_vectors[word]
    
dpt_store_centroid /= len(dpt_store_words)

In [21]:
with open("../data/taxonomy.txt") as f:
    products = f.read().splitlines()[1:]

In [22]:
top_taxons = set()
for product in products:
    top_taxon = product.split(">")[0]
    if top_taxon[-1] == " ":
        top_taxon = top_taxon[:-1]
    if top_taxon not in top_taxons:
        top_taxons.add(top_taxon)
        print(top_taxon)

Animals & Pet Supplies
Apparel & Accessories
Arts & Entertainment
Baby & Toddler
Business & Industrial
Cameras & Optics
Electronics
Food, Beverages & Tobacco
Furniture
Hardware
Health & Beauty
Home & Garden
Luggage & Bags
Mature
Media
Office Supplies
Religious & Ceremonial
Software
Sporting Goods
Toys & Games
Vehicles & Parts


In [23]:
unneeded_categories = {"Mature",     # Add any other categories not reflective of the real product list
                       "Vehicles & Parts",
                       "Arts & Entertainment",
                       "Cameras & Optics",
                       "Religious & Ceremonial",
                       "Vehicles & Parts",
                       "Animals and Pet Supplies"}

In [24]:
top_taxons -= unneeded_categories

In [29]:
taxon_dict = {}

for taxon in top_taxons:
    taxon_dict[taxon] = {}
    taxon_dict[taxon]["counter"] = Counter()
    taxon_dict[taxon]["glove_vector"] = np.zeros(300)
    taxon_dict[taxon]["total_count"] = 0

total = 0

for taxon in products:
    top_taxon = taxon.split(">")[0]
    if top_taxon[-1] == " ":
        top_taxon = top_taxon[:-1]
    if top_taxon in top_taxons:
        taxon = list(set(taxon.replace("&", "").replace(">", "").split()))
        for word in taxon:
            word = nlp(word)[0].lemma_
            if taxon_dict[top_taxon]["counter"][word] <= 10:
                if word in glove_vectors and cosine_similarity(dpt_store_centroid, glove_vectors[word]) > 0.1:
                    taxon_dict[top_taxon]["counter"][word] += 1
                    taxon_dict[top_taxon]["total_count"] += 1
                    taxon_dict[top_taxon]["glove_vector"] += glove_vectors[word]

for taxon in taxon_dict:
    taxon_dict[taxon]["glove_vector"] /= taxon_dict[taxon]["total_count"]
    taxon_dict[taxon] = taxon_dict[taxon]["glove_vector"]

In [ ]:
new_taxon_dict = {}

for k, v in taxon_dict.items():
    new_taxon_dict[k] = list(v)

new_taxon_dict

In [ ]:
with open("dpt_store_centroids.txt", 'w') as f:
    f.write("{")
    for key, value in new_taxon_dict.items(): 
        f.write('"%s":%s,\n' % (key, value))
    f.write("}")